# Credit Risk Resampling Techniques

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

# Read the CSV into DataFrame

In [4]:
# Load the data
file_path = Path('Resources/lending_data.csv')
df = pd.read_csv(file_path)
df.head()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,own,52800,0.431818,5,1,22800,low_risk
1,8400.0,6.692,own,43600,0.311927,3,0,13600,low_risk
2,9000.0,6.963,rent,46100,0.349241,3,0,16100,low_risk
3,10700.0,7.664,own,52700,0.430740,5,1,22700,low_risk
4,10800.0,7.698,mortgage,53000,0.433962,5,1,23000,low_risk


In [5]:
#Binary encode 'homeowner' column in dataframe
df = pd.get_dummies(df, columns=['homeowner'])

In [6]:
#Check binary encoded encoded dataframe 
df.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status,homeowner_mortgage,homeowner_own,homeowner_rent
0,10700.0,7.672,52800,0.431818,5,1,22800,low_risk,0,1,0
1,8400.0,6.692,43600,0.311927,3,0,13600,low_risk,0,1,0
2,9000.0,6.963,46100,0.349241,3,0,16100,low_risk,0,0,1
3,10700.0,7.664,52700,0.430740,5,1,22700,low_risk,0,1,0
4,10800.0,7.698,53000,0.433962,5,1,23000,low_risk,1,0,0


# Split the Data into Training and Testing

In [7]:
# Create our features
X = df.drop(columns=['loan_status'])

# Create our target
y = df['loan_status']

In [8]:
# Check the balance of our target values
y.value_counts()

low_risk     75036
high_risk     2500
Name: loan_status, dtype: int64

In [9]:
# Create X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test= train_test_split(X, y, random_state=1)
X_train.shape

(58152, 10)

In [10]:
#Check balance of low risk vs. high risk
Counter(y_train)

Counter({'low_risk': 56271, 'high_risk': 1881})

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [11]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [12]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)

In [13]:
# Scale the training and testing data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Simple Logistic Regression

In [14]:
#Train the Simple Logistic Regression using train data
from sklearn.linear_model import LogisticRegression
slr_model = LogisticRegression(solver='lbfgs', random_state=1)
slr_model.fit(X_train_scaled, y_train)

LogisticRegression(random_state=1)

In [15]:
# Calculated the balanced accuracy score
slr_y_pred = slr_model.predict(X_test_scaled)
slr_acc_score = balanced_accuracy_score(y_test, slr_y_pred)
print(f"Balanced Accuracy Score : {slr_acc_score}")

Balanced Accuracy Score : 0.9889115309798473


In [16]:
# Display the confusion matrix
slr_confusion_matrix = confusion_matrix(y_test, slr_y_pred)
slr_confusion_matrix_df = pd.DataFrame(
    slr_confusion_matrix, index=["Actual 0", "Actual 1"],
    columns=["Predicted 0", "Predicted 1"]
)
display(slr_confusion_matrix_df)

,Predicted 0,Predicted 1
Actual 0,609,10
Actual 1,113,18652


In [17]:
# Print the imbalanced classification report
print("Classification Report")
print(classification_report_imbalanced(y_test, slr_y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.84      0.98      0.99      0.91      0.99      0.98       619
   low_risk       1.00      0.99      0.98      1.00      0.99      0.98     18765

avg / total       0.99      0.99      0.98      0.99      0.99      0.98     19384



# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the following steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [18]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=1)
ros_X_resampled, ros_y_resampled = ros.fit_resample(X_train_scaled, y_train)


# View the count of target classes with Counter
Counter(ros_y_resampled)

Counter({'low_risk': 56271, 'high_risk': 56271})

In [19]:
# Train the Logistic Regression model using the resampled data
ros_model = LogisticRegression(solver='lbfgs', random_state=1)
ros_model.fit(ros_X_resampled, ros_y_resampled)

LogisticRegression(random_state=1)

In [20]:
# Calculated the balanced accuracy score
ros_y_pred = ros_model.predict(X_test_scaled)
ros_acc_score = balanced_accuracy_score(y_test, ros_y_pred)
print(f"Balanced Accuracy Score : {ros_acc_score}")

Balanced Accuracy Score : 0.9934649587814939


In [21]:
# Display the confusion matrix
ros_confusion_matrix = confusion_matrix(y_test, ros_y_pred)
ros_confusion_matrix_df = pd.DataFrame(
    ros_confusion_matrix, index=["Actual 0", "Actual 1"],
    columns=["Predicted 0", "Predicted 1"]
)
display(ros_confusion_matrix_df)

,Predicted 0,Predicted 1
Actual 0,615,4
Actual 1,124,18641


In [22]:
# Print the imbalanced classification report
print("Classification Report")
print(classification_report_imbalanced(y_test, ros_y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.83      0.99      0.99      0.91      0.99      0.99       619
   low_risk       1.00      0.99      0.99      1.00      0.99      0.99     18765

avg / total       0.99      0.99      0.99      0.99      0.99      0.99     19384



### SMOTE Oversampling

In [23]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE

smote_X_resampled, smote_y_resampled = SMOTE(random_state=1, sampling_strategy=1.0).fit_resample(X_train_scaled, y_train)

# View the count of target classes with Counter
Counter(smote_y_resampled)

Counter({'low_risk': 56271, 'high_risk': 56271})

In [24]:
# Train the Logistic Regression model using the resampled data
smote_model = LogisticRegression(solver='lbfgs', random_state=1)
smote_model.fit(smote_X_resampled, smote_y_resampled)

LogisticRegression(random_state=1)

In [25]:
# Calculated the balanced accuracy score
smote_y_pred = smote_model.predict(X_test_scaled)
smote_acc_score = balanced_accuracy_score(y_test, smote_y_pred)
print(f"Balanced Accuracy Score : {smote_acc_score}")

Balanced Accuracy Score : 0.9934649587814939


In [26]:
# Display the confusion matrix
smote_confusion_matrix = confusion_matrix(y_test, smote_y_pred)
smote_confusion_matrix_df = pd.DataFrame(
    smote_confusion_matrix, index=["Actual 0", "Actual 1"],
    columns=["Predicted 0", "Predicted 1"]
)
display(smote_confusion_matrix_df)

,Predicted 0,Predicted 1
Actual 0,615,4
Actual 1,124,18641


In [27]:
# Print the imbalanced classification report
print("Classification Report")
print(classification_report_imbalanced(y_test, smote_y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.83      0.99      0.99      0.91      0.99      0.99       619
   low_risk       1.00      0.99      0.99      1.00      0.99      0.99     18765

avg / total       0.99      0.99      0.99      0.99      0.99      0.99     19384



# Undersampling

In this section, you will test an undersampling algorithm to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [28]:
# Resample the data using the ClusterCentroids resampler
from imblearn.under_sampling import ClusterCentroids

cc = ClusterCentroids(random_state=1)
cc_X_resampled, cc_y_resampled = cc.fit_resample(X_train_scaled, y_train)

# View the count of target classes with Counter
Counter(cc_y_resampled)

Counter({'high_risk': 1881, 'low_risk': 1881})

In [29]:
# Train the Logistic Regression model using the resampled data
cc_model = LogisticRegression(solver='lbfgs', random_state=1)
cc_model.fit(cc_X_resampled, cc_y_resampled)

LogisticRegression(random_state=1)

In [30]:
# Calculate the balanced accuracy score
cc_y_pred = cc_model.predict(X_test_scaled)
cc_acc_score = balanced_accuracy_score(y_test, cc_y_pred)
print(f"Balanced Accuracy Score : {cc_acc_score}")

Balanced Accuracy Score : 0.9929503031930944


In [31]:
# Display the confusion matrix
cc_confusion_matrix = confusion_matrix(y_test, cc_y_pred)
cc_confusion_matrix_df = pd.DataFrame(
    cc_confusion_matrix, index=["Actual 0", "Actual 1"],
    columns=["Predicted 0", "Predicted 1"]
)
display(cc_confusion_matrix_df)

,Predicted 0,Predicted 1
Actual 0,614,5
Actual 1,113,18652


In [32]:
# Print the imbalanced classification report
print("Classification Report")
print(classification_report_imbalanced(y_test, cc_y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.84      0.99      0.99      0.91      0.99      0.99       619
   low_risk       1.00      0.99      0.99      1.00      0.99      0.99     18765

avg / total       0.99      0.99      0.99      0.99      0.99      0.99     19384



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [33]:
# Resample the training data with SMOTEENN
from imblearn.combine import SMOTEENN

sm = SMOTEENN(random_state=1)
sm_X_resampled, sm_y_resampled = sm.fit_resample(X_train_scaled, y_train)

# View the count of target classes with Counter
Counter(sm_y_resampled)

Counter({'high_risk': 55509, 'low_risk': 55937})

In [34]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression

sm_model = LogisticRegression(solver='lbfgs', random_state=1)
sm_model.fit(sm_X_resampled, sm_y_resampled)

LogisticRegression(random_state=1)

In [35]:
# Calculate the balanced accuracy score
sm_y_pred = sm_model.predict(X_test_scaled)
sm_acc_score = balanced_accuracy_score(y_test, sm_y_pred)
print(f"Balanced Accuracy Score : {sm_acc_score}")

Balanced Accuracy Score : 0.9934649587814939


In [36]:
# Display the confusion matrix
sm_confusion_matrix = confusion_matrix(y_test, sm_y_pred)
sm_confusion_matrix_df = pd.DataFrame(
    sm_confusion_matrix, index=["Actual 0", "Actual 1"],
    columns=["Predicted 0", "Predicted 1"]
)
display(sm_confusion_matrix_df)

,Predicted 0,Predicted 1
Actual 0,615,4
Actual 1,124,18641


In [37]:
# Print the imbalanced classification report
print("Classification Report")
print(classification_report_imbalanced(y_test, sm_y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.83      0.99      0.99      0.91      0.99      0.99       619
   low_risk       1.00      0.99      0.99      1.00      0.99      0.99     18765

avg / total       0.99      0.99      0.99      0.99      0.99      0.99     19384



# Final Questions

1. Which model had the best balanced accuracy score?

   Naive random oversampling, SMOTE and SMOTEEN score the highest with identical balanced accuracy scores.


2. Which model had the best recall score?

    All scored 99%.

3. Which model had the best geometric mean score?

    All scored 99%.
